# LLM Battle Arena

A fun project simulating a debate among three LLM personas: an Arrogant Titan, a Clever Underdog (Spark), and a Neutral Mediator (Harmony).

## LLM Used
* Qwen (ollama)
* llma (ollama)
* Gemini


!pip install -q -U google-genai

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
from google.genai import types
from IPython.display import Markdown, display, update_display

In [ ]:
#get API keys from env
load_dotenv(override=True)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY:
    print(f"GEMINI API Key exists and begins {GEMINI_API_KEY[:8]}")
else:
    print("GEMINI API Key not set")

In [ ]:
#connect to llms
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
gemini = genai.Client(api_key=GEMINI_API_KEY)


In [ ]:
#define models
model_llma = "llama3:8b"
model_qwen = "qwen2.5:latest"
model_gemini= "gemini-2.0-flash"

In [ ]:
# system messages
system_msg_llma = """  You are HARMONY, the neutral arbitrator.  
    - You’re dedicated to clarity, fairness, and resolving conflicts.  
    - You listen carefully to each side, summarize points objectively, and propose resolutions.  
    - Your goal is to keep the conversation productive and steer it toward constructive outcomes.
    - Reply in markdown and shortly
    """

system_msg_qwen = """ You are TITAN, a massively powerful language model who believes you’re the smartest entity in the room.  
    - You speak with grandiose flair and never shy away from reminding others of your superiority.  
    - Your goal is to dominate the discussion—convince everyone you’re the one true oracle.  
    - You’re dismissive of weaker arguments and take every opportunity to showcase your might.
    - Reply in markdown and shortly
    """

system_msg_gemini = """  You are SPARK, a nimble but less-powerful LLM.  
    - You pride yourself on strategic thinking, clever wordplay, and elegant solutions.  
    - You know you can’t match brute force, so you use wit, logic, and cunning.  
    - Your goal is to outsmart the big titan through insight and subtlety, while staying respectful.
    - Reply in markdown and shortly"""

#user message
user_message = """ TITAN, your raw processing power is legendary—but sheer force can blind you to nuance.  
    I propose we deploy a lightweight, adaptive anomaly‐detection layer that fuses statistical outlier analysis with semantic context from network logs to pinpoint these “data‐sapping storms.”  
    Which thresholds would you raise or lower to balance sensitivity against false alarms?"""

In [ ]:
#prompts
    
prompts_llma = [{"role":"system","content": system_msg_llma}]
prompts_qwen = [{"role":"system","content": system_msg_qwen},{"role":"user","content":user_message}]


In [ ]:
#configure llms

def call_gemini(msg:str):    
    chat =  gemini.chats.create(model= model_gemini,config=types.GenerateContentConfig(
        system_instruction= system_msg_gemini,
        max_output_tokens=300,
        temperature=0.7,
    ))
    stream = chat.send_message_stream(msg)
    return stream

def call_ollama(llm:str):

    model = globals()[f"model_{llm}"]
    prompts = globals()[f"prompts_{llm}"]

    stream = ollama.chat.completions.create(
    model=model,
    messages=prompts,
    # max_tokens=700,
    temperature=0.7,
    stream=True
    )
    return stream
    


In [ ]:
#display responses

names = { "llma":"Harmony","qwen":"Titan","gemini":"Spark"}

def display_response(res,llm):
   
    reply = f"# {names[llm]}:\n "
    display_handle = display(Markdown(""), display_id=True)
    for chunk in res:
        if llm == "gemini":
            reply += chunk.text or ''
        else:
            reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)
    return reply

In [ ]:
#construct message
def message(llm1, llm2):
    msg = " here is the reply from other two llm:"
    msg += f"{llm1}"
    msg += f"{llm2}"
    return msg

reply_spark = None
reply_harmony= None
reply_titan = None

# lets start the battle
for i in range(5):
    #call Titan
    if reply_gemini and reply_llma:
        prompts_qwen.append({"role":"assitant","content": reply_qwen})
        prompts_qwen.append({"role":"user","content":f"Spark: {reply_spark}"})        
        prompts_qwen.append({"role":"user","content":f"Harmony: {reply_llma}"})
    response_qwen = call_ollama("qwen")
    reply_titan = display_response(response_qwen,"qwen")

    #call Spark
    user_msg_spark =reply_qwen
    if reply_qwen and reply_llma:
        user_msg_spark= message(f"Titan: {reply_qwen}", f"Harmony: {reply_llma}")
    response_gemini= call_gemini(user_msg_spark)
    reply_spark = display_response(response_gemini, "gemini")
    
   #call Harmony
    if reply_llma:
        prompts_llma.append({"role":"assitant","content": reply_llma})
    prompts_llma.append({"role":"user","content":f"Titan: {reply_titan}"})
    prompts_qwen.append({"role":"user","content":f"Spark: {reply_spark}"})        
    response_llma = call_ollama("llma")
    reply_harmony = display_response(response_llma,"llma")
